Estructuras de control. Diccionarios. Archivos JSON. ¿A dónde voy en las vacaciones de invierno? Gratis…

Este proyecto consiste en permitir el registro y consulta de eventos gratuitos que se realicen dentro del territorio argentino. Está dirigido a dos tipos de usuarios.

Para aquellos que deseen dar a conocer un evento, indicando día, hora y lugar de realización, como así también el nombre y descripción del evento.
Para aquellos que deseen consultar la oferta de eventos en su municipio o un listado de eventos a realizarse en un periodo de tiempo.
El programa iniciará ofreciendo un menú (Registro/Consulta). Registro solicitará los datos necesarios para generar un archivo JSON a partir del diseño de diccionarios con la siguiente forma: { “nombre_evento”: “Festival por aniversario del Club Atlético Libertad”, "descripcion": “Evento familiar con músicos locales”, "dia": 12, "mes": “julio”, "hora": 15, "provincia": “Salta”, "localidad": “Tartagal”, }

Los datos “provincia” y “localidad” deben ser extraídos de las correspondientes APIs. Es decir, debe mostrarse al usuario una lista de provincias y a partir de su elección, una lista de las localidades de esa provincia.

Consulta ofrecerá otro menú (Eventos en mi localidad/Todos los eventos/Eventos entre dos fechas).

Para “Eventos en mi localidad”, nuevamente accederemos a la información de las APIs para conocer provincia y localidad del usuario y luego buscaremos dentro del archivo JSON ya generado todos los eventos vinculados.

Para “Todos los eventos” simplemente debemos listar completamente la información del archivo JSON generado.

Para “Eventos entre dos fechas” solicitamos al usuario día y mes de inicio y día y mes de fin del intervalo que desea conocer y se realizará una búsqueda dentro del JSON creado que cumpla con los requisitos pedidos.

Para realizar esta actividad contamos con las siguientes URLs

https://apis.datos.gob.ar/georef/api/provincias?campos=id,nombre

https://apis.datos.gob.ar/georef/api/municipios?provincia=22&c ampos=id,nombre&max=100

In [1]:
import requests
import json

# Variables globales
nombre_archivo = 'eventos.json'
meses = {
    1: 'enero',
    2: 'febrero',
    3: 'marzo',
    4: 'abril',
    5: 'mayo',
    6: 'junio',
    7: 'julio',
    8: 'agosto',
    9: 'septiembre',
    10: 'octubre',
    11: 'noviembre',
    12: 'diciembre'
}


def main():
    print("¡BIENVENIDOS!")
    while True:
        print("\n---------------- Menú ----------------")
        print("1. Registro de un nuevo evento")
        print("2. Consulta de eventos ya registrados")
        print("0. Salir")
        opcion = input("Seleccione una opción: ")

        if opcion == "1":
            registrar_evento()
        elif opcion == "2":
          while True:
            print("\n---------- Menú de Consulta ----------")
            print("A. Consulta de eventos en mi localidad")
            print("B. Consulta de todos los eventos")
            print("C. Consulta de eventos entre dos fechas")
            print("0. Volver")
            subopcion = input("Seleccione una opción: ").upper()

            if subopcion == "A":
                buscar_por_localidad()
            elif subopcion == "B":
                listar_eventos()
            elif subopcion == "C":
                buscar_por_fechas()
            elif subopcion == "0":
               break
            else:
                print("Opción inválida. Intente nuevamente.")
        elif opcion == "0":
            break
        else:
            print("Opción inválida. Intente nuevamente.")


def registrar_evento():
    """
    Registra un nuevo evento solicitando información al usuario y lo guarda en un archivo JSON.
    """
    nombre_evento = input("\nIngrese el nombre del evento: ")
    descripcion = input("Ingrese la descripción del evento: ")
    dia = ingreso_dia_valido()
    numero_mes = ingreso_numero_mes_valido()
    mes = meses[numero_mes]
    hora = ingreso_hora_valido()

    provincias = obtener_provincias()
    mostrar_provincias(provincias)

    fin_consulta = False
    while True:
        id_provincia = input(f"Ingrese el id de provincia: ")

        if id_provincia in provincias:
            localidades = obtener_localidades(id_provincia)
            if localidades:
                mostrar_localidades(localidades, id_provincia, provincias[id_provincia])

                while True:
                    id_localidad = input(f"Ingrese el id de la localidad: ")

                    if id_localidad in localidades:
                        evento = {
                            "nombre_evento": nombre_evento,
                            "descripcion": descripcion,
                            "dia": dia,
                            "mes": mes,
                            "hora": hora,
                            "provincia": provincias[id_provincia],
                            "localidad": localidades[id_localidad]
                        }

                        eventos = deserializar()
                        eventos.append(evento)
                        serializar(eventos)

                        print("\n¡Evento registrado con éxito! ✔️")
                        fin_consulta = True
                        break
                    else:
                        print("Id de localidad inválido. Intente nuevamente.")
            else:
                print("No se encontraron localidades en la provincia especificada. Por favor, elija otra provincia.")
        else:
            print("Id de provincia inválido. Intente nuevamente.")
        if fin_consulta:
            break


def ingreso_dia_valido():
    """
    Solicita al usuario ingresar un día válido para un evento.
    """
    while True:
        try:
            dia = int(input("Ingrese el día en el que se hará el evento (ejemplo: 12): "))
            if 1 <= dia <= 31:
                break
            else:
                print("El día ingresado no es válido. Intente nuevamente.")
        except ValueError:
            print("Ingrese un número entero para el día.")
    return dia


def ingreso_numero_mes_valido():
    """
    Solicita al usuario ingresar un numero de mes válido para un evento.
    """
    while True:
        try:
            numero_mes = int(input("Ingrese el mes en el que se hará el evento (ejemplo: 1 para enero): "))
            if 1 <= numero_mes <= 12:
                break
            else:
                print("El número de mes ingresado no es válido. Intente nuevamente.")
        except ValueError:
            print("Ingrese un número entero para el mes.")
    return numero_mes


def ingreso_hora_valido():
    """
    Solicita al usuario ingresar una hora válida para un evento.
    """
    while True:
        try:
            hora = int(input("Ingrese la hora de inicio del evento (ejemplo: 15): "))
            if 0 <= hora <= 23:
                break
            else:
                print("La hora ingresada no es válida. Intente nuevamente.")
        except ValueError:
            print("Ingrese un número entero para la hora.")
    return hora


def obtener_provincias():
    """
    Obtiene un diccionario con las provincias y sus identificadores desde una API.
    """
    response = requests.get("https://apis.datos.gob.ar/georef/api/provincias?campos=id,nombre").text
    objeto = json.loads(response)
    lista = objeto["provincias"]

    provincias = dict()

    for diccionario in lista:
        provincias[diccionario["id"]] = diccionario["nombre"]

    return provincias


def obtener_localidades(id):
    """
    Obtiene un diccionario con las localidades y sus identificadores de una provincia específica desde una API.
    """
    response = requests.get(f"https://apis.datos.gob.ar/georef/api/municipios?provincia={id}&campos=id,nombre&max=100").text
    objeto = json.loads(response)
    lista = objeto["municipios"]

    localidades = dict()

    for diccionario in lista:
        localidades[diccionario["id"]] = diccionario["nombre"]

    return localidades


def mostrar_provincias(provincias):
    """
    Muestra una lista de provincias con sus identificadores.
    """
    print("\nLista de provincias:")
    for id, provincia in provincias.items():
        print(f"- {id}. {provincia}")


def mostrar_localidades(localidades, id_provincia, nombre_provincia):
    """
    Muestra una lista de localidades con sus identificadores de una provincia específica.
    """
    print(f"\nLista de localidades de {nombre_provincia}:")
    for id, localidad in localidades.items():
        print(f"- {id}. {localidad}")


def deserializar():
    """
    Deserializa el contenido de un archivo JSON y devuelve su contenido.
    Si el archivo especificado no existe, se creará uno nuevo con una lista vacía como contenido inicial.
    """
    try:
        with open(nombre_archivo, 'r') as f:
            contenido = json.load(f)
    except FileNotFoundError:
        contenido = list()
        with open(nombre_archivo, 'w') as f:
            json.dump(contenido, f)
    return contenido


def serializar(contenido):
    """
    Serializa un contenido en formato JSON y lo guarda en un archivo.
    """
    with open(nombre_archivo, 'w') as f:
        json.dump(contenido, f, indent=4, ensure_ascii=False)


def buscar_por_localidad():
    """
    Realiza una búsqueda de eventos por localidad.
    """
    print("\n----- Búsqueda por Localidad -----")

    provincias = obtener_provincias()
    mostrar_provincias(provincias)

    fin_consulta = False
    while True:
        id_provincia = input(f"Ingrese el id de provincia: ")

        if id_provincia in provincias:
            localidades = obtener_localidades(id_provincia)
            if localidades:
                mostrar_localidades(localidades, id_provincia, provincias[id_provincia])

                while True:
                    id_localidad = input(f"Ingrese el id de la localidad: ")

                    if id_localidad in localidades:
                        eventos = deserializar()
                        eventos_en_localidad = []

                        if eventos:
                            encontrado = False
                            for evento in eventos:
                                if evento["provincia"] == provincias[id_provincia] and evento["localidad"] == localidades[id_localidad]:
                                    mostrar_evento(evento)
                                    encontrado = True

                            if not encontrado:
                                print("\nNo se encontraron eventos en la localidad especificada.")
                        else:
                            print("\nNo hay eventos registrados.")
                        fin_consulta = True
                        break
                    else:
                        print("Id de localidad inválido. Intente nuevamente.")
            else:
                print("\nNo se encontraron localidades en la provincia especificada.")
                fin_consulta = True
                break
        else:
            print("Id de provincia inválido. Intente nuevamente.")
        if fin_consulta:
            break


def listar_eventos():
    """
    Muestra un listado de eventos almacenados en un archivo JSON.
    """
    print("\n-------- Listado de eventos --------")
    eventos = deserializar()

    if eventos != []:
        for evento in eventos:
            mostrar_evento(evento)
    else:
        print("\nNo hay eventos registrados.")


def buscar_por_fechas():
    """
    Realiza una búsqueda de eventos entre dos fechas y muestra los detalles de los eventos encontrados.
    """
    print("\n----- Búsqueda entre dos fechas -----")
    eventos = deserializar()

    if eventos != []:
        print("FECHA INICIO:")
        dia_ini = ingreso_dia_valido()
        mes_ini = ingreso_numero_mes_valido()
        print("FECHA FIN:")
        dia_fin = ingreso_dia_valido()
        mes_fin = ingreso_numero_mes_valido()

        fecha_inicio = (2023, mes_ini, dia_ini)
        fecha_fin = (2023, mes_fin, dia_fin)

        meses_invertidos = {v.lower(): k for k, v in meses.items()}

        encontrado = False
        for evento in eventos:
            fecha_evento = (2023, meses_invertidos[evento['mes'].lower()], evento['dia'])

            if fecha_inicio <= fecha_evento <= fecha_fin:
                mostrar_evento(evento)
                encontrado = True

        if not encontrado:
            print(f"\nNo se encontraron eventos entre {dia_ini} {meses[mes_ini]} y {dia_fin} {meses[mes_fin]}.")
    else:
        print("\nNo hay eventos registrados.")


def mostrar_evento(evento):
    """
    Muestra los detalles de un evento proporcionado como parámetro.
    """
    print(f'\nEVENTO: "{evento["nombre_evento"]}"')
    print("Descripción:", evento["descripcion"])
    print("Fecha:", evento["dia"], evento["mes"])
    print("Hora:", evento["hora"])
    print("Provincia:", evento["provincia"])
    print("Localidad:", evento["localidad"])


if __name__ == "__main__":
    main()

¡BIENVENIDOS!

---------------- Menú ----------------
1. Registro de un nuevo evento
2. Consulta de eventos ya registrados
0. Salir
Seleccione una opción: 1

Ingrese el nombre del evento: Feria folclorica
Ingrese la descripción del evento: Evento cultural
Ingrese el día en el que se hará el evento (ejemplo: 12): 06
Ingrese el mes en el que se hará el evento (ejemplo: 1 para enero): 088
El número de mes ingresado no es válido. Intente nuevamente.
Ingrese el mes en el que se hará el evento (ejemplo: 1 para enero): 08
Ingrese la hora de inicio del evento (ejemplo: 15): 10

Lista de provincias:
- 54. Misiones
- 74. San Luis
- 70. San Juan
- 30. Entre Ríos
- 78. Santa Cruz
- 62. Río Negro
- 26. Chubut
- 14. Córdoba
- 50. Mendoza
- 46. La Rioja
- 10. Catamarca
- 42. La Pampa
- 86. Santiago del Estero
- 18. Corrientes
- 82. Santa Fe
- 90. Tucumán
- 58. Neuquén
- 66. Salta
- 22. Chaco
- 34. Formosa
- 38. Jujuy
- 02. Ciudad Autónoma de Buenos Aires
- 06. Buenos Aires
- 94. Tierra del Fuego, Ant

In [ ]:
import requests
import json

# Variables globales
nombre_archivo = 'eventos.json'
meses = {
    1: 'enero',
    2: 'febrero',
    3: 'marzo',
    4: 'abril',
    5: 'mayo',
    6: 'junio',
    7: 'julio',
    8: 'agosto',
    9: 'septiembre',
    10: 'octubre',
    11: 'noviembre',
    12: 'diciembre'
}
def deserializar():
    """
    Deserializa el contenido de un archivo JSON y devuelve su contenido.
    Si el archivo especificado no existe, se creará uno nuevo con una lista vacía como contenido inicial.
    """
    try:
        with open(nombre_archivo, 'r') as f:
            contenido = json.load(f)
    except FileNotFoundError:
        contenido = list()
        with open(nombre_archivo, 'w') as f:
            json.dump(contenido, f)
    return contenido


def serializar(contenido):
    """
    Serializa un contenido en formato JSON y lo guarda en un archivo.
    """
    with open(nombre_archivo, 'w') as f:
        json.dump(contenido, f, indent=4, ensure_ascii=False)

# Función para obtener la lista de provincias desde la API
def obtener_provincias():
    url = "https://apis.datos.gob.ar/georef/api/provincias?campos=id,nombre"
    response = requests.get(url)
    data = response.json()
    provincias = {provincia["id"]: provincia["nombre"] for provincia in data["provincias"]}
    return provincias

# Función para obtener la lista de localidades de una provincia desde la API
def obtener_localidades(provincia_id):
    url = f"https://apis.datos.gob.ar/georef/api/municipios?provincia={provincia_id}&campos=id,nombre&max=100"
    response = requests.get(url)
    data = response.json()
    localidades = {localidad["id"]: localidad["nombre"] for localidad in data["municipios"]}
    return localidades

# Función para registrar un evento en el archivo JSON
def registrar_evento():
    nombre_evento = input("Ingrese el nombre del evento: ")
    descripcion = input("Ingrese la descripción del evento: ")
    dia = int(input("Ingrese el día de realización del evento: "))
    mes = input("Ingrese el mes de realización del evento: ")
    hora = int(input("Ingrese la hora de realización del evento: "))

    provincias = obtener_provincias()
    for id_provincia, nombre_provincia in provincias.items():
        print(f"{id_provincia}: {nombre_provincia}")

    provincia_id = input("Seleccione el ID de la provincia donde se realizará el evento: ")
    localidades = obtener_localidades(provincia_id)
    for id_localidad, nombre_localidad in localidades.items():
        print(f"{id_localidad}: {nombre_localidad}")

    localidad_id = input("Seleccione el ID de la localidad donde se realizará el evento: ")

    evento = {
        "nombre_evento": nombre_evento,
        "descripcion": descripcion,
        "dia": dia,
        "mes": mes,
        "hora": hora,
        "provincia": provincias[provincia_id],
        "localidad": localidades[localidad_id]
    }
    eventos = deserializar()
    eventos.append(evento)
    serializar(eventos)
    print("Evento registrado correctamente.")

# Función para consultar eventos desde el archivo JSON
def consultar_eventos():
    print("1. Eventos en mi localidad")
    print("2. Todos los eventos")
    print("3. Eventos entre dos fechas")

    opcion = int(input("Seleccione una opción: "))

    if opcion == 1:
        provincias = obtener_provincias()
        for id_provincia, nombre_provincia in provincias.items():
            print(f"{id_provincia}: {nombre_provincia}")

        provincia_id = input("ID de la Provincia: ")
        if provincia_id in provincias:
          localidades = obtener_localidades(provincia_id)
          for id_localidad, nombre_localidad in localidades.items():
            print(f"{id_localidad}: {nombre_localidad}")

          localidad_id = input("ID de la Localidad: ")

        # Obtener las localidades y provincia desde la API
          if id_localidad in localidades:
            eventos = deserializar()
            eventos_localidad = []
            print("entro if localidades")
            if eventos:
              print("if eventos")
              encontrado = False
              for linea in eventos:
                if eventos[provincias] == provincias[provincia_id] and eventos[localidades] == localidades[localidad_id]:
                  mostrar_evento(evento)
                  eventos.append(evento)
                  encontrado = True

              if not encontrado:
                print("No se encontraron Eventos en tu localidad: ")
            else:
              print("No hay eventos registrados")
          else:
            print("Id de localidad invalido.")
    elif opcion == 2:
      eventos = deserializar()
      if eventos != []:
        print("Todos los eventos: ")
        for evento in eventos:
          mostrar_evento(evento)
      else:
          print("No hay eventos registrados.")
    elif opcion == 3:
      eventos = deserializar()
      if eventos != []:
          # Obtener fechas de inicio y fin del usuario
        dia_inicio = int(input("Día de inicio (número): "))
        mes_inicio = int(input("Mes de Inicio: "))
        dia_fin = int(input("Día de fin(número): "))
        mes_fin = int((input("Mes de fin: ")))

        fecha_inicio = (2023, mes_inicio, dia_inicio)
        fecha_fin = (2023, mes_fin, dia_fin)

        meses = {v.lower(): k for k, v in meses.items()}

        for evento in eventos:
          fecha_evento = (2023, meses[evento['mes'].lower()], evento['dia'])

          if fecha_inicio <= fecha_evento <= fecha_fin:
            print("Eventos entre las fechas especificadas: ")
            mostrar_evento(evento)
            encontrado = True
        if not encontrado:
            print("No se encontraron eventos en el intervalo especificado.")
      else:
        print("No hay eventos registrados.")

# Función principal que muestra el menú principal
def main():
    while True:
        print("Menú Principal")
        print("1. Registro de evento")
        print("2. Consulta de eventos")
        print("3. Salir")
        opcion = int(input("Seleccione una opción: "))

        if opcion == 1:
            registrar_evento()
        elif opcion == 2:
            consultar_eventos()
        elif opcion == 3:
            break
        else:
            print("Opción inválida.")

def mostrar_evento(evento):
    """
    Muestra los detalles de un evento proporcionado como parámetro.
    """
    print(f'\nEVENTO: "{evento["nombre_evento"]}"')
    print("Descripción:", evento["descripcion"])
    print("Fecha:", evento["dia"], evento["mes"])
    print("Hora:", evento["hora"])
    print("Provincia:", evento["provincia"])
    print("Localidad:", evento["localidad"])

if __name__ == "__main__":
    main()



Menú Principal
1. Registro de evento
2. Consulta de eventos
3. Salir
Seleccione una opción: 1
Ingrese el nombre del evento: folc
Ingrese la descripción del evento: music
Ingrese el día de realización del evento: 19
Ingrese el mes de realización del evento: 8
Ingrese la hora de realización del evento: 22
54: Misiones
74: San Luis
70: San Juan
30: Entre Ríos
78: Santa Cruz
62: Río Negro
26: Chubut
14: Córdoba
50: Mendoza
46: La Rioja
10: Catamarca
42: La Pampa
86: Santiago del Estero
18: Corrientes
82: Santa Fe
90: Tucumán
58: Neuquén
66: Salta
22: Chaco
34: Formosa
38: Jujuy
02: Ciudad Autónoma de Buenos Aires
06: Buenos Aires
94: Tierra del Fuego, Antártida e Islas del Atlántico Sur
Seleccione el ID de la provincia donde se realizará el evento: 66
660084: La Merced
660035: Las Lajitas
660252: Tolar Grande
660056: Cafayate
660077: Cerrillos
660133: Embarcación
660280: Río Piedras
660119: General Güemes
660343: Santa Victoria Este
660364: Campo Quijano
660378: Angastaco
660126: Aguaray
6

TypeError: ignored

In [ ]:

import requests
import json

def obtener_provincias():
    url = "https://apis.datos.gob.ar/georef/api/provincias?campos=id,nombre"
    response = requests.get(url)
    data = response.json()
    provincias = {provincia["id"]: provincia["nombre"] for provincia in data["provincias"]}
    return provincias

def obtener_localidades(provincia):
    url = f"https://apis.datos.gob.ar/georef/api/municipios?provincia={provincia}&campos=id,nombre&max=100"
    response = requests.get(url)
    data = response.json()
    localidades = {localidad["id"]: localidad["nombre"] for localidad in data["municipios"]}
    return localidades

def registrar():
    nombre_evento= input("Ingrese el nombre del evento: ")
    descripcion= input("Ingrese la descripción del evento: ")
    dia=int(input("Ingrese el día de realización del evento: "))
    mes= input("Ingrese el mes de realización del evento: ")
    hora= int(input("Ingrese la hora de realización del evento: "))

    provincias = obtener_provincias()
    for id_provincia, nombre_provincia in provincias.items():
        print(f"{id_provincia}: {nombre_provincia}")

    provincia_id = int(input("Seleccione el ID de la provincia donde se realizará el evento: "))
    localidades = obtener_localidades(provincia_id)
    for id_localidad, nombre_localidad in localidades.items():
        print(f"{id_localidad} {nombre_localidad}")

    localidad_id = int(input("Seleccione el ID de la localidad donde se realizará el evento: "))

    datos_evento = {
        "nombre_evento": nombre_evento,
        "descripcion": descripcion,
        "dia": dia,
        "mes": mes,
        "hora": hora,
        "provincia": provincias[provincia_id],
        "localidad": localidades[localidad_id]
    }

    with open("eventos.json", "a") as archivo:
      json.dump(datos_evento, archivo)
      archivo.write("\n")

    print("Evento registrado correctamente.")

def consultar_eventos():
    print("1. Eventos en mi localidad")
    print("2. Todos los eventos")
    print("3. Eventos entre dos fechas")
    opcion = int(input("Seleccione una opción: "))

    if opcion == 1:
        # Implementar consulta de eventos en la localidad del usuario
        pass
    elif opcion == 2:
        # Implementar listado de todos los eventos
        pass
    elif opcion == 3:
        # Implementar búsqueda de eventos entre dos fechas
        pass
    else:
        print("Opción inválida.")

def menu():
  while True:
    print("--------------------------------------------------")
    print("       Eventos gratuitos                          ")
    print("--------------------------------------------------")
    print("1.Consultar eventos gratuitos disponibles")
    print("2.Registrar eventos gratuitos")
    print("3.Salir")
    opcion=int(input("Ingrese la opción deseada: "))


    if opcion==1:
      consultar_eventos()
    elif opcion==2:
      registrar()
    else:
        if opcion==3:
          break
        else:
          print ("la opción ingresada no es correcta")


if __name__ == "__main__":
    menu()


--------------------------------------------------
       Eventos gratuitos                          
--------------------------------------------------
1.Consultar eventos gratuitos disponibles
2.Registrar eventos gratuitos
3.Salir
Ingrese la opción deseada: 1
1. Eventos en mi localidad
2. Todos los eventos
3. Eventos entre dos fechas
Seleccione una opción: 1
--------------------------------------------------
       Eventos gratuitos                          
--------------------------------------------------
1.Consultar eventos gratuitos disponibles
2.Registrar eventos gratuitos
3.Salir
Ingrese la opción deseada: 2
Ingrese el nombre del evento: puthon
Ingrese la descripción del evento: programacion
Ingrese el día de realización del evento: 12
Ingrese el mes de realización del evento: 08
Ingrese la hora de realización del evento: 23
54: Misiones
74: San Luis
70: San Juan
30: Entre Ríos
78: Santa Cruz
62: Río Negro
26: Chubut
14: Córdoba
50: Mendoza
46: La Rioja
10: Catamarca
42: La Pa

KeyError: ignored